# Rice Balst Disease Prediction using AI


In [1]:
import os
import glob
import pandas as pd
#set working directory
#find all csv files in the folder
#use glob pattern matching -> extension = 'csv'
#save result in list -> all_filenames
extension = 'csv'
#all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#print(all_filenames)
all_filenames = ["train_14d.csv", "test_14d.csv"]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( r"combined_csv.csv", index=False, encoding='utf-8-sig')

In [2]:
import pandas as pd

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [ ]:
# Read data file
BC_df = pd.read_csv("combined_csv.csv")
print("The shape of our dataset :"+str(BC_df.shape))
BC_df.head()

In [ ]:
BC_df.isnull().any()

In [ ]:
BC_df.drop("Unnamed: 0",axis=1,inplace=True)
#data.drop("label", axis=1, inplace=True)

In [ ]:
BC_df.shape

In [ ]:
diagnosis_set = set(BC_df['label'])
print(diagnosis_set)

In [ ]:
# Checking for imbalance 
BC_count = BC_df['label'].value_counts()
print(BC_count)


In [ ]:
X = BC_df.drop(columns={'label'})
X

In [ ]:
y = BC_df['label']
y

In [ ]:
#using SMOTE from imblearn
import imblearn
from imblearn.combine import SMOTEENN
smote = SMOTEENN(random_state=0)
X_res, y_res = smote.fit_resample(X,y)
print('Resampled dataset shape %s' % y_res.value_counts())

In [ ]:
print(y_res)

In [ ]:
enoder = LabelEncoder().fit(y_res)
y_res = enoder.transform(y_res)
# BC_df.head()
y_res

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_res,y_res,
                                test_size=0.2,random_state=0)
print('Shape of the training set:'+" "+str(x_train.shape))
print('Size of the training set:'+" "+str(x_train.shape[0]))

print('Shape of the testing set:'+" "+str(x_test.shape))
print('Size of the testing set:'+" "+str(x_test.shape[0]))

In [ ]:
scaler = StandardScaler().fit(x_train)
train_scaled = scaler.transform(x_train)
test_scaled = scaler.transform(x_test)

In [ ]:
model = Sequential([
    Dense(32 ,activation='relu',input_shape=(173,)),
    Dense(32 ,activation='relu'),
    Dense(1,activation='sigmoid')
])
model.summary()

In [ ]:
model.compile(optimizer='sgd',
             loss='binary_crossentropy',
             metrics=['accuracy'])
model.fit(train_scaled , y_train ,epochs=100,batch_size=32 )

In [ ]:
model.save('model_RiceBlast.h5')

In [ ]:
pretrained_model = keras.models.load_model('./model_RiceBlast.h5')
eval_score = pretrained_model.evaluate(test_scaled,y_test)
print('The loss of the evaluation :'+' '+str(eval_score[0]))
print('The accuracy of the evaluation :'+' '+str(eval_score[1]))

In [ ]:
y_pred = pretrained_model.predict_classes(test_scaled)
y_pred

In [ ]:
target_list=['no disease','disease']
report=classification_report(y_true=y_test,y_pred=y_pred,output_dict = True,target_names=target_list)
print(report)

In [ ]:
CV = confusion_matrix(y_true=y_test,y_pred=y_pred)
sns.heatmap(CV,annot=True )
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()